In [9]:
pip install zep-python

  Obtaining dependency information for zep-python from https://files.pythonhosted.org/packages/e2/8b/3935777a859db5a0b4850ba5273c3d427e4e28384fbbca86dd4a48bae714/zep_python-1.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<0.25.0,>=0.24.0 from https://files.pythonhosted.org/packages/ec/91/e41f64f03d2a13aee7e8c819d82ee3aa7cdc484d18c0ae859742597d5aa0/httpx-0.24.1-py3-none-any.whl.metadata
  Using cached httpx-0.24.1-py3-none-any.whl.metadata (7.4 kB)
  Obtaining dependency information for httpcore<0.18.0,>=0.15.0 from https://files.pythonhosted.org/packages/94/2c/2bde7ff8dd2064395555220cbf7cba79991172bf5315a07eb3ac7688d9f1/httpcore-0.17.3-py3-none-any.whl.metadata
  Using cached httpcore-0.17.3-py3-none-any.whl.metadata (18 kB)
Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
Using cached httpcore-0.17.3-py3-none-any.whl (74 kB)
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.5
    Uninstalling httpcore-1.0.5:
      Successfully

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.14.0 requires pydantic>=2.0, but you have pydantic 1.10.14 which is incompatible.
langchain-mistralai 0.1.6 requires httpx<1,>=0.25.2, but you have httpx 0.24.1 which is incompatible.
langchain-mistralai 0.1.6 requires tokenizers<0.16.0,>=0.15.1, but you have tokenizers 0.13.3 which is incompatible.


In [26]:
from uuid import uuid4

from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ZepMemory
from langchain.retrievers import ZepRetriever
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import OpenAI

# Set this to your Zep server URL
ZEP_API_URL = "http://localhost:8000"

session_id = str(uuid4())  # This is a unique identifier for the user

In [27]:
openai_key = 'sk-proj-qKoq6YIhC1N9z8ZzFTp6T3BlbkFJl50imb4r13ddqDS2KT4d'
search = WikipediaAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description=(
            "useful for when you need to search online for answers. You should ask"
            " targeted questions"
        ),
    ),
]

# Set up Zep Chat History
memory = ZepMemory(
    session_id=session_id,
    url=ZEP_API_URL,
    #api_key=zep_api_key,
    memory_key="chat_history",
)

# Initialize the agent
llm = OpenAI(temperature=0, openai_api_key=openai_key)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [28]:
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
        "metadata": {"foo": "bar"},
    },
]

for msg in test_history:
    memory.chat_memory.add_message(
        (
            HumanMessage(content=msg["content"])
            if msg["role"] == "human"
            else AIMessage(content=msg["content"])
        ),
        metadata=msg.get("metadata", {}),
    )




In [31]:
from zep_python import (
    MemorySearchPayload,
    ZepClient
)
client = ZepClient(ZEP_API_URL)
# This uniquely identifies the user's session
session_id = "39f2ee75-fd92-4594-a3a2-0b5ee337732e"

# Initialize the Zep client before running this code
search_payload = MemorySearchPayload(
    text="Is Lauren Olamina a character in a book?",
    search_scope="summary", # This could be messages or summary
    #search_type="mmr", # remove this if you'd prefer not to rerank results
    #mmr_lambda=0.5, # tune diversity vs relevance
)

search_results = await client.memory.asearch_memory(session_id, search_payload)

for search_result in search_results:
    # Uncomment for message search
    # print(search_result.messsage.dict())
    print(search_result.summary.dict())


TypeError: MemoryClient.asearch_memory() missing 1 required positional argument: 'search_payload'